# Demo Notebook for Express Model Loading and Result Table templates

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['schema_name'] = "nnfabrik_konsti_playground"
schema = dj.schema("nnfabrik_konsti_playground")

dj.config["display.limit"]=5

import torch
import numpy as np
import time

import nnfabrik
from nnfabrik import main, builder
from nnfabrik.main import *

import nnvision

Connecting kwilleke@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306
Schema `nnfabrik_konsti_playground`



## Instantiating the DataInfo table

In [3]:
from nnvision.tables.from_nnfabrik import TrainedModel, DataInfo

In [4]:
DataInfo().populate()

In [5]:
DataInfo()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,data_info Dictionary of data_keys and i/o information,fabrikant_name Name of the contributor that added this entry,datainfo_ts UTZ timestamp at time of insertion
nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,=BLOB=,kwilleke,2020-04-28 23:11:53
nnvision.datasets.monkey_static_loader,4b2fd27381ac30177f4f1f6805c678ee,=BLOB=,kwilleke,2020-04-28 23:11:53
nnvision.datasets.monkey_static_loader,ad2646906c82ada12ec442a4f8905fe8,=BLOB=,kwilleke,2020-04-28 23:11:53


#### here's a glimpse of whats in the data info 

In [6]:
DataInfo().fetch("data_info")

array([{'3646924284383': {'input_dimensions': (128, 1, 16, 16), 'input_channels': 1, 'output_dimension': 2, 'img_mean': 112.48829, 'img_std': 64.01505}, '3651417046988': {'input_dimensions': (128, 1, 16, 16), 'input_channels': 1, 'output_dimension': 2, 'img_mean': 112.48829, 'img_std': 64.01505}},
       {'3575478071862': {'input_dimensions': (128, 1, 15, 15), 'input_channels': 1, 'output_dimension': 15, 'img_mean': 111.63308, 'img_std': 44.28877}, '3572282892581': {'input_dimensions': (128, 1, 15, 15), 'input_channels': 1, 'output_dimension': 18, 'img_mean': 111.63308, 'img_std': 44.28877}},
       {'3631896544452': {'input_dimensions': (128, 1, 11, 11), 'input_channels': 1, 'output_dimension': 32, 'img_mean': 113.92844, 'img_std': 63.936275}, '3632669014376': {'input_dimensions': (128, 1, 11, 11), 'input_channels': 1, 'output_dimension': 21, 'img_mean': 113.92844, 'img_std': 63.936275}}],
      dtype=object)

In [7]:
TrainedModel()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,.V1_Model_gauss.CSRF19_V4,0.0785511,=BLOB=,kwilleke,2020-04-23 13:14:28
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,2000,.V1_Model_gauss.CSRF19_V4,0.0708413,=BLOB=,kwilleke,2020-04-23 15:18:24
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,4b2fd27381ac30177f4f1f6805c678ee,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,.V1_Model_gauss.PlosCB19_V1,0.187103,=BLOB=,kwilleke,2020-04-23 13:22:17
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,4b2fd27381ac30177f4f1f6805c678ee,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,2000,.V1_Model_gauss.PlosCB19_V1,0.168166,=BLOB=,kwilleke,2020-04-23 15:18:41
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,ad2646906c82ada12ec442a4f8905fe8,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,.V1_Model_gauss.CSRF19_V1_40_6,0.23656,=BLOB=,kwilleke,2020-04-23 13:14:04


# Define a key

In [8]:
example_key = TrainedModel.fetch("KEY")[0]

# Model loading without building the dataloader

In [10]:
before = time.time()
model = (TrainedModel&example_key).load_model(include_dataloader=False, include_state_dict=True)
print("elapsed time: {}s".format(time.time()-before))

elapsed time: 0.5123541355133057s


# Get a Unit Table

In [11]:
from nnvision.tables.main import MonkeyExperiment

Schema name: nnfabrik_konsti_playground


In [12]:
MonkeyExperiment.Units()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,data_key,unit_id,unit_index
nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,3646924284383,2,0
nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,3646924284383,3,1
nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,3651417046988,3,0
nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,3651417046988,5,1
nnvision.datasets.monkey_static_loader,4b2fd27381ac30177f4f1f6805c678ee,3572282892581,0,0


# Fill Result Tables

### Import Result functions

In [13]:
from nnvision.utility.measures import get_correlations, get_avg_correlations, get_FEV

### Import the template of the score table

In [14]:
from nnfabrik.template import ScoringBase

# Instantiate result tables - One table per measure

In [15]:
@schema
class TrainCorrelation(ScoringBase):
    trainedmodel_table = TrainedModel
    unit_table = MonkeyExperiment.Units
    scoring_function = staticmethod(get_correlations)
    scoring_dataset = "train"
    scoring_attribute = "train_correlation"

TrainCorrelation().populate(example_key, display_progress=True)

0it [00:00, ?it/s]


In [16]:
TrainCorrelation()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,train_correlation A template for a computed score of a trained model,fabrikant_name Name of the contributor that added this entry,train_correlation_ts UTZ timestamp at time of insertion
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,0.0827348,kwilleke,2020-04-28 23:45:12


In [17]:
TrainCorrelation.UnitScore()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,data_key,unit_id,train_correlation A template for a computed unit score
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,3646924284383,2,0.17294
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,3646924284383,3,0.148804
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,3651417046988,3,0.0111443
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,3651417046988,5,-0.00194716


In [18]:
@schema
class ValidationCorrelation(ScoringBase):
    trainedmodel_table = TrainedModel
    unit_table = MonkeyExperiment.Units
    scoring_function = staticmethod(get_correlations)
    scoring_dataset = "validation"
    scoring_attribute = "validation_correlation"
    
ValidationCorrelation().populate(example_key, display_progress=True)

0it [00:00, ?it/s]


In [19]:
@schema
class TestCorrelation(ScoringBase):
    trainedmodel_table = TrainedModel
    unit_table = MonkeyExperiment.Units
    scoring_function = staticmethod(get_correlations)
    scoring_dataset = "test"
    scoring_attribute = "test_correlation"

TestCorrelation().populate(example_key, display_progress=True)

0it [00:00, ?it/s]


In [20]:
@schema
class AverageCorrelation(ScoringBase):
    trainedmodel_table = TrainedModel
    unit_table = MonkeyExperiment.Units
    scoring_function = staticmethod(get_avg_correlations)
    scoring_attribute = "avg_correlation"

AverageCorrelation().populate(example_key, display_progress=True)

0it [00:00, ?it/s]


In [21]:
@schema
class FEVe(ScoringBase):
    trainedmodel_table = TrainedModel
    unit_table = MonkeyExperiment.Units
    scoring_function = staticmethod(get_FEV)
    scoring_dataset = "test"
    scoring_attribute = "feve"
    
FEVe().populate(example_key, display_progress=True)

0it [00:00, ?it/s]


In [22]:
FEVe()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,feve A template for a computed score of a trained model,fabrikant_name Name of the contributor that added this entry,feve_ts UTZ timestamp at time of insertion
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,-0.0338058,kwilleke,2020-04-28 22:37:44


In [23]:
FEVe().UnitScore()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,data_key,unit_id,feve A template for a computed unit score
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,3646924284383,2,0.0658183
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,3646924284383,3,-0.00561118
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,3651417046988,3,-0.0909464
nnvision.models.se_core_full_gauss_readout,28d96f527b5cc4579c9d03859c69ce56,nnvision.datasets.monkey_static_loader,117249699f3051cf435e6254d2972d02,nnvision.training.nnvision_trainer,93ddfe9d207ad5075416759970e51fb9,1000,3651417046988,5,-0.104484
